<a href="https://colab.research.google.com/github/njeruhi/GPT/blob/main/GPT_testbed_on_tiktoken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tiktoken

In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.nn import functional as F

In [3]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 2500
eval_interval = 50
learning_rate = 3e-4
device = 'cuda'
eval_iters = 10
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

In [4]:
with open("tiny-shakespeare.txt", 'r') as f:
    data = f.read()

In [5]:
import tiktoken
enc = tiktoken.get_encoding('cl100k_base')
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4")

In [6]:
dataset = enc.encode(data)
dataset[:25]

[5451,
 47317,
 512,
 10438,
 584,
 10570,
 904,
 4726,
 11,
 6865,
 757,
 6604,
 382,
 2460,
 512,
 96945,
 11,
 6604,
 382,
 5451,
 47317,
 512,
 2675,
 527,
 682]

In [7]:
dataset = torch.tensor(dataset, dtype=torch.long)
print(dataset.dtype, dataset.shape)
print(dataset[:20])

torch.int64 torch.Size([301829])
tensor([ 5451, 47317,   512, 10438,   584, 10570,   904,  4726,    11,  6865,
          757,  6604,   382,  2460,   512, 96945,    11,  6604,   382,  5451])


In [8]:
vocab_size = len(dataset)
vocab_size

301829

In [9]:
n = int(0.9*len(dataset))
train_data = dataset[:n]
val_data = dataset[n:]

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([ 5451, 47317,   512, 10438,   584, 10570,   904,  4726,    11])

In [11]:
torch.manual_seed(1337)
batch_size = 4

In [12]:
def get_batch(split):

    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [15]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [16]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [17]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [18]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        B,T, C = logits.shape

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
    #For each block within the batch, this method generates the most probable token(s) to come next
    #For simple use, pass a batch of only one block to this method in order to simulate "ChatGPT" like interaction of "prompt" and "answer"
        # idx is (B, T) array of indices in the current context
        idx.to(device)
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [19]:
model = GPTLanguageModel()
m = model.to(device)
print("Training is using " + device)
print(f"Model parameter device: {next(model.parameters()).device}")
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


Training is using cuda
Model parameter device: cuda:0
242.750213 M parameters
step 0: train loss 12.6466, val loss 12.6703
step 50: train loss 8.6202, val loss 9.3712
step 100: train loss 8.0111, val loss 8.5212
step 150: train loss 7.7599, val loss 8.4041
step 200: train loss 7.7320, val loss 7.8145
step 250: train loss 7.3126, val loss 7.9679
step 300: train loss 7.4016, val loss 7.8457
step 350: train loss 7.1934, val loss 7.4342
step 400: train loss 7.6296, val loss 7.1122
step 450: train loss 7.7567, val loss 7.6546
step 500: train loss 7.2040, val loss 7.5950
step 550: train loss 7.3109, val loss 7.5415
step 600: train loss 7.1057, val loss 7.6542
step 650: train loss 7.3290, val loss 7.3010
step 700: train loss 6.5189, val loss 7.2354
step 750: train loss 6.9842, val loss 7.2467
step 800: train loss 6.5065, val loss 7.0065
step 850: train loss 6.9567, val loss 7.2212
step 900: train loss 6.5943, val loss 6.8981
step 950: train loss 6.3751, val loss 6.8007
step 1000: train loss 6

In [20]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)

In [21]:
pred_tokens=m.generate(context, max_new_tokens=50)[0]

In [23]:
encoded_tokens = pred_tokens.tolist()
encoded_tokens


[0,
 58909,
 1203,
 1427,
 4265,
 369,
 26236,
 35482,
 1983,
 121344,
 25,
 1218,
 79,
 10443,
 1600,
 596,
 2800,
 449,
 5389,
 512,
 74209,
 11,
 358,
 25229,
 1781,
 813,
 3663,
 7220,
 28534,
 2450,
 2586,
 26236,
 3663,
 345,
 51,
 13,
 7477,
 315,
 263,
 7020,
 11,
 345,
 8140,
 2908,
 11,
 21846,
 3472,
 2092,
 48969,
 856,
 54614]